In [1]:
!pip install scipy==1.10.1 scikit-image==0.19.3 vit_keras==0.1.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.9/13.9 MB 59.3 MB/s eta 0:00:00
  Obtaining dependency information for validators from https://files.pythonhosted.org/packages/3a/0c/785d317eea99c3739821718f118c70537639aa43f96bfa1d83a71f68eaf6/validators-0.22.0-py3-none-any.whl.metadata
  Attempting uninstall: scipy
    Found existing installation: scipy 1.11.3
    Uninstalling scipy-1.11.3:
      Successfully uninstalled scipy-1.11.3
  Attempting uninstall: scikit-image
    Found existing installation: scikit-image 0.21.0
    Uninstalling scikit-image-0.21.0:
      Successfully uninstalled scikit-image-0.21.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cuml 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cuml 23.8.0 requires dask==2023.7.1, but you have dask 2023.

In [2]:
import os
import cv2
import random
import shutil
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras as keras
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, Model
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from PIL import Image
from sklearn.preprocessing import LabelEncoder
import numpy as np
from vit_keras import vit, utils
import lime
import skimage
import shap
import pandas as pd
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
from lime.lime_image import LimeImageExplainer

<frozen importlib._bootstrap>:241: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject
/opt/conda/lib/python3.10/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [3]:
train_dir = '/kaggle/input/dogs-cats-images/dataset/training_set'
test_dir = '/kaggle/input/dogs-cats-images/dataset/test_set'

classes = os.listdir(train_dir)[0:2]
classes

['dogs', 'cats']

In [4]:
resize_size = 256
crop_size = 224

def preprocess_image(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = tf.image.resize(image, [resize_size, resize_size], method=tf.image.ResizeMethod.BILINEAR) #크기 조절
    image = tf.image.central_crop(image, central_fraction=crop_size / resize_size) #중앙 224x224
    image = tf.math.divide(image, 255.0) #normalize
    mean = [0.485, 0.456, 0.406]
    std = [0.229, 0.224, 0.225]
    image = (image - mean) / std #다 normalize

    return image


def resize_and_crop(image, resize_size=256, crop_size=224):
    # Resize with bilinear interpolation
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    resized_image = tf.image.resize(image, [resize_size, resize_size], method=tf.image.ResizeMethod.BILINEAR)
    cropped_image = tf.image.central_crop(resized_image, central_fraction=crop_size / resize_size)
    image = tf.math.divide(cropped_image, 255.0) #normalize

    return image

In [5]:
X = [] #Image
y = [] #class
Original_X = []
X_name = []
for i in classes:
    for data_dir in [train_dir, test_dir]:
        folderPath = os.path.join(data_dir,i)
        for j in tqdm(os.listdir(folderPath)):
            img = cv2.imread(os.path.join(folderPath,j)) #이미지 읽기
            O_img = resize_and_crop(img)
            Original_X.append(O_img)
            img = preprocess_image(img) #전처리
            X.append(img) #X list 넣고
            y.append(i) # y list
            X_name.append(j)
X = np.array(X)
y = np.array(y)
y = tf.keras.utils.to_categorical([classes.index(label) for label in y])
Original_X = np.array(Original_X)

100%|██████████| 1000/1000 [00:13<00:00, 73.45it/s]


In [6]:
for model_name in ['vgg']:
    model = keras.models.load_model('/kaggle/input/dog-and-cat-classifier/'+model_name+"_best_model.h5", compile=False)
    model.compile(optimizer='SGD', loss='categorical_crossentropy', metrics=['accuracy'])
    X_train,xx,y_train,yy = train_test_split(X, y, test_size=0.2,random_state=42)
    X_val,X_test,y_val,y_test = train_test_split(xx, yy, test_size=0.5,random_state=42)
    y_pred = model.predict(X_test)
    y_pred_single_label = np.argmax(y_pred, axis=1)
    y_test_single_label = np.argmax(y_test, axis=1)
    precision = precision_score(y_test_single_label, y_pred_single_label, average='macro')
    recall = recall_score(y_test_single_label, y_pred_single_label, average='macro')
    f1 = f1_score(y_test_single_label, y_pred_single_label, average='macro')
    accuracy = accuracy_score(y_test_single_label, y_pred_single_label)
    precision = round(precision, 3)
    recall = round(recall, 3)
    f1 = round(f1, 3)
    accuracy = round(accuracy, 3)
    print(str(model_name))
    print(model_name, ":", precision, recall, f1, accuracy)

32/32 [==============================] - 15s 166ms/step
vgg
vgg : 0.976 0.976 0.976 0.976


In [7]:
import matplotlib.pyplot as plt
import cv2

def plot_images(original, temp, mask, model_name, model_type, index_to_use, predicted_class, real):
    
    save_paths = [f'{model_name}/{model_type}/original/', f'{model_name}/{model_type}/superpixel/', f'{model_name}/{model_type}/pos_neg/']
    
    for save_path in save_paths:
        if not os.path.exists(save_path):
            os.makedirs(save_path)

    plt.imshow(original)
    plt.axis('off')
    plt.savefig(os.path.join(f'{model_name}/{model_type}/original/', f'{index_to_use}_predict_{predicted_class}_real_{real}.jpg'), bbox_inches='tight', pad_inches=0)
    plt.close()
    
    # Lime이 생성한 이미지 표시 (temp)
    # plt.imshow(temp)
    # plt.set_title('Lime Image')
    # plt.axis('off')
    # plt.show()

    # Lime Mask를 원본 이미지에 적용하여 긍부정 시각화
    masked_positive = np.copy(original)
    masked_positive[mask <= 0] = 0  # 부정 부분은 검은색으로 설정
    
    plt.imshow(masked_positive)
    plt.axis('off')
    plt.savefig(os.path.join(f'{model_name}/{model_type}/superpixel/', f'{index_to_use}_predict_{predicted_class}_real_{real}.jpg'), bbox_inches='tight', pad_inches=0)
    plt.close()

    # Lime Mask를 원본 이미지에 적용하여 긍부정 시각화
    masked_negative = np.zeros_like(original)
    masked_negative[mask < 0] = [255, 0, 0]

    # Create a new image for positive parts (green color)
    masked_positive = np.zeros_like(original)
    masked_positive[mask > 0] = [0, 255, 0]

    # Combine the positive and negative images
    combined_image = original + masked_negative + masked_positive

    # Display the result with larger size
    plt.imshow(combined_image)
    plt.axis('off')
    plt.savefig(os.path.join(f'{model_name}/{model_type}/pos_neg/', f'{index_to_use}_predict_{predicted_class}_real_{real}.jpg'), bbox_inches='tight', pad_inches=0)
    plt.close()

In [8]:
import sys
import io
for index_to_use in range(0,5):
    selected_image = X[index_to_use]
    selected_image = np.expand_dims(selected_image, axis=0)
    prediction = model.predict(selected_image)
    predicted_class = np.argmax(prediction)
    real = np.argmax(y[index_to_use])
    original_stdout = sys.stdout
    sys.stdout = io.StringIO()
    explainer = lime.lime_image.LimeImageExplainer(feature_selection='auto')
    explanation = explainer.explain_instance(selected_image[0], model.predict, top_labels=1, hide_color=0, num_samples=1000)
    temp, mask = explanation.get_image_and_mask(explanation.top_labels[0], positive_only=False, num_features=10, hide_rest=False)
    sys.stdout = original_stdout
    plot_images(Original_X[index_to_use], temp, mask, model_name, 'base_model', index_to_use, predicted_class, real)

1/1 [==============================] - 1s 534ms/step


  0%|          | 0/1000 [00:00<?, ?it/s]

1/1 [==============================] - 0s 22ms/step


  0%|          | 0/1000 [00:00<?, ?it/s]

1/1 [==============================] - 0s 20ms/step


  0%|          | 0/1000 [00:00<?, ?it/s]

1/1 [==============================] - 0s 22ms/step


  0%|          | 0/1000 [00:00<?, ?it/s]

1/1 [==============================] - 0s 22ms/step


  0%|          | 0/1000 [00:00<?, ?it/s]

In [9]:
def preprocess_for_attack(image):
    #resize, crop까지는 이미 이전 과정에서 진행했으므로
    mean = [0.485, 0.456, 0.406]
    std = [0.229, 0.224, 0.225]
    image = (image - mean) / std #다 normalize

    return image

In [10]:
def ifgsm_attack(model, image, label, epsilon=0.001, num_iter=30, clip_min=0.0, clip_max=1.0):
    adv_image = tf.identity(image)

    for _ in range(num_iter):
        with tf.GradientTape() as tape:
            tape.watch(adv_image)
            prediction = model(preprocess_for_attack(adv_image))
            loss = tf.keras.losses.sparse_categorical_crossentropy(label, prediction)

        gradient = tape.gradient(loss, adv_image)
        perturbation = epsilon * tf.sign(gradient)

        adv_image = tf.clip_by_value(adv_image + perturbation, clip_min, clip_max)

    adv_image_np = adv_image.numpy().squeeze()  # Squeeze to remove channel dimension

    # Lime explanation
    original_stdout = sys.stdout
    sys.stdout = io.StringIO()

    explainer = lime.lime_image.LimeImageExplainer(feature_selection='auto')
    explanation = explainer.explain_instance(adv_image_np, model.predict, top_labels=1, hide_color=0, num_samples=1000)
    temp, mask = explanation.get_image_and_mask(explanation.top_labels[0], positive_only=False, num_features=10, hide_rest=False)

    sys.stdout = original_stdout
    
    # Assuming you have model_name, index_to_use, predicted_class, real defined somewhere
    plot_images(adv_image.numpy().squeeze(), temp, mask, model_name, 'adv', index_to_use, predicted_class, real)

    return adv_image, prediction

In [11]:
#Le, L. D., Fu, H., Xu, X., Liu, Y., Xu, Y., Du, J., ... & Goh, R. (2022, September). An Efficient Defending Mechanism Against Image Attacking on Medical Image Segmentation Models. In MICCAI Workshop on Resource-Efficient Medical Image Analysis (pp. 65-74). Cham: Springer Nature Switzerland.

In [12]:
save_paths = [f'{model_name}/adv/attack/']
for save_path in save_paths:
    if not os.path.exists(save_path):
        os.makedirs(save_path)
            
for index_to_use in range(0,5):
    selected_image = X[index_to_use]
    selected_image = np.expand_dims(selected_image, axis=0)
    prediction = model.predict(selected_image)
    predicted_class = np.argmax(prediction)
    real_class = np.argmax(y[index_to_use])
    preprocess_input = tf.keras.applications.vgg16.preprocess_input

    image = Original_X[index_to_use]
    image = tf.expand_dims(image, axis=0)  # Add batch dimension
    label = np.argmax(y[index_to_use])

    original_prediction = model(preprocess_for_attack(image))
    adv_image, adv_prediction = ifgsm_attack(model, image, label)

    # Extract top-1 predictions
    original_top1_class = tf.argmax(original_prediction, axis=1)[0:5]
    adv_top1_class = tf.argmax(adv_prediction, axis=1)[0:5]
    
    plt.imshow(adv_image.numpy().squeeze())
    plt.axis('off')
    plt.savefig(os.path.join(f'{model_name}/adv/attack/', f'{index_to_use}_predict_{predicted_class}_advpredict_{adv_top1_class[0]}_real_{real}.jpg'), bbox_inches='tight', pad_inches=0)
    plt.close()
    
'''
    # Display the results
    plt.figure(figsize=(15, 6))
    plt.subplot(1, 4, 1)
    plt.imshow(image.numpy().squeeze())
    plt.title('Original Image')

    plt.subplot(1, 4, 2)
    plt.text(0, 0.5, f'Top-1 Prediction: Class {original_top1_class}', fontsize=12)
    plt.axis('off')

    plt.subplot(1, 4, 3)
    plt.imshow(adv_image.numpy().squeeze())
    plt.title('Adversarial Image')

    plt.subplot(1, 4, 4)
    plt.text(0, 0.5, f'Top-1 Prediction: Class {adv_top1_class}', fontsize=12)
    plt.axis('off')
    plt.show()
'''

1/1 [==============================] - 0s 21ms/step


  0%|          | 0/1000 [00:00<?, ?it/s]

1/1 [==============================] - 0s 20ms/step


  0%|          | 0/1000 [00:00<?, ?it/s]

1/1 [==============================] - 0s 25ms/step


...

1/1 [==============================] - 0s 21ms/step


  0%|          | 0/1000 [00:00<?, ?it/s]

1/1 [==============================] - 0s 21ms/step


  0%|          | 0/1000 [00:00<?, ?it/s]

"\n    # Display the results\n    plt.figure(figsize=(15, 6))\n    plt.subplot(1, 4, 1)\n    plt.imshow(image.numpy().squeeze())\n    plt.title('Original Image')\n\n    plt.subplot(1, 4, 2)\n    plt.text(0, 0.5, f'Top-1 Prediction: Class {original_top1_class}', fontsize=12)\n    plt.axis('off')\n\n    plt.subplot(1, 4, 3)\n    plt.imshow(adv_image.numpy().squeeze())\n    plt.title('Adversarial Image')\n\n    plt.subplot(1, 4, 4)\n    plt.text(0, 0.5, f'Top-1 Prediction: Class {adv_top1_class}', fontsize=12)\n    plt.axis('off')\n    plt.show()\n"

In [13]:
import os
import shutil
from IPython.display import FileLink

# Change the current working directory to '/kaggle/working'
os.chdir('/kaggle/working')

# Define the name of the tar.gz archive
archive_name = 'kaggle_working.tar.gz'

# Create a tar.gz archive of the entire directory
shutil.make_archive(archive_name.replace('.tar.gz', ''), 'gztar', root_dir='.', base_dir='.')

# Display the link to download the archive
FileLink(archive_name)

/kaggle/working/kaggle_working.tar.gz